In [1]:
'''
This is Homework 5 for AMATH 581
Tianbo Zhang 1938501
'''
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
from scipy.fft import fft2, ifft2
from scipy.integrate import solve_ivp
from scipy.linalg import kron

In [2]:
# Set up parameters
Lx = 20    # spatial domain of x
Ly = 20    # spatial domain of y
nx = 64   # number of discretization points in x
ny = 64   # number of discretization points in y
N = nx * ny   # elements in reshaped initial condition
tspan = np.arange(0, 4.5, 0.5)    # time span
beta = 1
D_1 = 0.1
D_2 = 0.1

# Set up domains
x2 = np.linspace(-Lx/2, Lx/2, nx+1) # x domain
x = x2[:nx]   
y2 = np.linspace(-Ly/2, Ly/2, ny+1) # y domain
y = y2[:ny]  
X, Y = np.meshgrid(x, y)  # make 2D

# Initial conditions
m = 1
U0 = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
V0 = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
U_hat = fft2(U0).flatten()
V_hat = fft2(V0).flatten()
init_conditions = np.concatenate([U_hat, V_hat])

# Coefficients
A_2 = lambda U, V: U**2 + V**2
lam = lambda A2: 1 - A2
omega = lambda A2: -beta * A2

# Define spectral k values
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = -(KX**2 + KY**2)

In [3]:
# FFT Method
n_fft = 64
def rhs_periodic(t, uv, n, D1, D2):
    Ut, Vt = uv[:n**2].reshape((n, n)), uv[n**2:].reshape((n, n))
    U = ifft2(Ut)
    V = ifft2(Vt)
    A2_val = A_2(U, V)
    lam_val = lam(A2_val)
    omega_val = omega(A2_val)
    dUdt = fft2(lam_val * U - omega_val * V) + D1 * K * Ut
    dVdt = fft2(omega_val * U + lam_val * V) + D2 * K * Vt
    return np.concatenate([dUdt.flatten(), dVdt.flatten()])

sol_periodic = solve_ivp(rhs_periodic, [tspan[0], tspan[-1]], init_conditions, t_eval=tspan, args = (n_fft,D_1, D_2,), method='RK45')
A1 = sol_periodic.y

In [4]:
# Part b: Chebyshev
n_cheb = 30
# Code from lecture (Chebyshev)
def cheb(N):
	if N==0: 
		D = 0.; x = 1.
	else:
		n = arange(0,N+1)
		x = cos(pi*n/N).reshape(N+1,1) 
		c = (hstack(( [2.], np.ones(N-1), [2.]))*(-1)**n).reshape(N+1,1)
		X = tile(x,(1,N+1))
		dX = X - X.T
		D = dot(c,1./c.T)/(dX+eye(N+1))
		D -= diag(sum(D.T,axis=0))
	return D, x.reshape(N+1)
    
D, x = cheb(n_cheb)
D = D/10 
D[n_cheb, :] = 0
D[0, :] = 0
D2 = np.dot(D, D)
I = np.eye(len(D2))
L = kron(I, D2) + kron(D2, I) 
x = x*10
y = x
X, Y = np.meshgrid(x, y)

# Initial conditions
m = 1
U0 = (np.tanh(np.sqrt(X**2+Y**2))*np.cos(m*np.angle(X+1j*Y)-(np.sqrt(X**2+Y**2)))).flatten()
V0 = (np.tanh(np.sqrt(X**2+Y**2))*np.sin(m*np.angle(X+1j*Y)-(np.sqrt(X**2+Y**2)))).flatten()
init_conditions_cheb = np.concatenate([U0, V0])

# Right-hand side for no-flux boundary conditions
def rhs_no_flux(t, uv, n, D1, D2, L):
    U1, V1 = uv[:(n+1)**2], uv[(n+1)**2:]
    U2, V2 = U1.reshape((n+1, n+1)), V1.reshape((n+1, n+1))
    A2_val = A_2(U2, V2)
    lam_val = lam(A2_val)
    omega_val = omega(A2_val)
    dUdt = lam_val * U2 - omega_val * V2 + D1 * (np.dot(L, U1).reshape(n+1, n+1))
    dVdt = omega_val * U2 + lam_val * V2 + D2 * (np.dot(L, V1).reshape(n+1, n+1))
    return np.concatenate([dUdt.flatten(), dVdt.flatten()])
sol_cheb = solve_ivp(rhs_no_flux, [tspan[0], tspan[-1]], init_conditions_cheb, t_eval=tspan, args = (n_cheb, D_1, D_2, L,), method='RK45')
A2 = sol_cheb.y